# Geospatial statistics of Covid-19 in Israel

The following analysis on the geospatial statistic properties of the Covid-19 epidemy in Israel is based on the official dataset released by the Government of Israel ([link](https://data.gov.il/dataset?q=COVID-19)).

## Imports

In [1]:
import pandas as pd

from pyproj import CRS, Transformer #Geodesic transformations

# Vizualization
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

## Read data

We get the actual raw data from the dataset:

In [2]:
from  dateutil.parser import parser

p = parser()

geo_data_raw = pd.read_excel('..\data\geographic-summary-per-day-2020-06-12.xlsx', usecols = ["accumulated_cases", "date", "town_code", "town"])
def show_cases_by_city(city_code):
    return geo_data_raw[geo_data_raw["town"].str.contains(city_code)]

interact(show_cases_by_city, city_code='')
plt.show()
print(geo_data_raw.dtypes)


interactive(children=(Text(value='', description='city_code'), Output()), _dom_classes=('widget-interact',))

town_code                     int64
date                 datetime64[ns]
accumulated_cases            object
town                         object
dtype: object


We load the list of towns (which we downloaded from the CBS website):

In [3]:
cities_df = pd.read_excel("../data/townsInfo.xlsx", usecols = ["שם יישוב", "קואורדינטות", 'תעתיק', "סמל יישוב"])
cities_df.rename(columns={'סמל יישוב':'town_code', 'שם יישוב':'town_name'}, inplace=True)
cities_df.head(10)

,town_name,town_code,תעתיק,קואורדינטות
0,אבו ג'ווייעד (שבט),967,ABU JUWEI'ID,2.040057e+09
1,אבו גוש,472,ABU GHOSH,2.105263e+09
2,אבו סנאן,473,ABU SINAN,2.160776e+09
3,אבו סריחאן (שבט),935,ABU SUREIHAN,1.865057e+09
4,אבו עבדון (שבט),958,ABU ABDUN,1.850058e+09
5,אבו עמאר (שבט),1042,ABU AMMAR,1.950057e+09
6,אבו עמרה (שבט),932,ABU AMRE,1.850057e+09
7,אבו קורינאת (יישוב),1342,ABU QUREINAT,1.963656e+09
8,אבו קורינאת (שבט),968,ABU QUREINAT,1.970056e+09
9,אבו רובייעה (שבט),966,ABU RUBEI'A,2.050057e+09


## Preprocess

First we create a table in which each line is a town, and each column contains the number of sick people by date.

In [4]:
geo_data = geo_data_raw.copy()
geo_data[['accumulated_cases']].fillna(value='0')
kk=geo_data['accumulated_cases'].str.match('<15', na=False)
geo_data.loc[kk, 'accumulated_cases'] = '0'
geo_data['accumulated_cases'] = geo_data['accumulated_cases'].astype(float)

def show_cases_by_city(city_code):
    return geo_data[geo_data["town"].str.contains(city_code)]

interact(show_cases_by_city, city_code='')
plt.show()
print(geo_data.dtypes)

interactive(children=(Text(value='', description='city_code'), Output()), _dom_classes=('widget-interact',))

town_code                     int64
date                 datetime64[ns]
accumulated_cases           float64
town                         object
dtype: object


Aggregate by twon and date combination:

In [5]:
geo_data_agg = geo_data.groupby(['town_code', 'date', 'town']).agg({'accumulated_cases':'sum'}).reset_index()
# display(geo_data_agg)
def show_cases_by_city(city_code):
    return geo_data_agg[geo_data_agg["town"].str.contains(city_code)]
interact(show_cases_by_city, city_code='')
plt.show()

interactive(children=(Text(value='', description='city_code'), Output()), _dom_classes=('widget-interact',))

Make it so for each town we have one row with historical data in columns:

In [6]:
geo_data_processed = geo_data_agg.pivot('town_code', 'date', 'accumulated_cases')
geo_data_processed.drop(index=0, inplace=True)
# display(geo_data_processed.index)
def show_cases_by_city(city_code):
    return geo_data_processed[geo_data_processed.index==int(city_code)]
interact(show_cases_by_city, city_code='0')
plt.show()

interactive(children=(Text(value='0', description='city_code'), Output()), _dom_classes=('widget-interact',))

Display kind of interactively:

In [7]:
# print(df.columns)
# pd.DataFrame(df.columns).merge(cities_df,on='town_code', how='left')

In [8]:
from bokeh.palettes import Spectral10
from bokeh.plotting import figure, output_file, show, output_notebook, reset_output
from bokeh.models import CategoricalColorMapper
from bokeh.models import ColumnDataSource
from datetime import datetime

df = geo_data_processed.sort_values(by='2020-06-12', ascending=False).head(10)
# display(df)
df.index = pd.DataFrame(df.index).merge(cities_df,on='town_code', how='inner')['town_name']
# display(df)
# df.index
# df.loc['town_code',:]
# df.columns = df.loc['City',:]
# df.drop(index=['City'], inplace=True)
# df['Date'] = pd.to_datetime(df.index, format= '%Y-%m-%d')
# display(df)

# source = ColumnDataSource(data=df)
dates_str = df.columns.to_list()
cities_names = df.index.tolist()

color_mapper = CategoricalColorMapper(factors=cities_names, palette=Spectral10)

p = figure(plot_width=800, plot_height=500, x_axis_type="datetime")
p.title.text = 'Click on legend entries to hide the corresponding lines'

x = dates_str

for irow,color in zip(df.iterrows(), Spectral10):
#     df = pd.DataFrame(data)
    row = irow[1]
    city_name = row.name
    y = row.to_list()
    p.line(x, y, line_width=2, alpha=0.8, legend_label=city_name, color=color)

p.legend.location = "top_left"
p.legend.click_policy="hide"

# output_file("interactive_legend.html", title="interactive_legend.py example")
reset_output()
output_notebook()

show(p)
# df

Loading BokehJS ...

Geolocate cities and associate names:

In [9]:
itm_crs = CRS.from_proj4("+proj=tmerc +lat_0=31.7343936111111 +lon_0=35.2045169444445 +k=1.0000067 +x_0=219529.584 +y_0=626907.39 +ellps=GRS80 +towgs84=-24.002400,-17.103200,-17.844400,-0.33077,-1.852690,1.669690,5.424800 +units=m +no_defs")
import numpy as np
wgs84_crs = CRS.from_epsg(4326)
transformer = Transformer.from_crs(itm_crs, wgs84_crs)

cities_df = pd.read_excel("../data/townsInfo.xlsx", usecols = ["שם יישוב", "קואורדינטות", 'תעתיק', "סמל יישוב"])
cities_df.dropna(axis=0, how='any', inplace=True)
cities_df.loc[:, "קואורדינטות"] = cities_df.loc[:, "קואורדינטות"].round(0).astype(np.int64).astype(str)
cities_df['x'] = cities_df.loc[:, "קואורדינטות"].str[0:5].astype(int)
cities_df['y'] = cities_df.loc[:, "קואורדינטות"].str[5:].astype(int)
lats, lons = transformer.transform(cities_df['x'].to_numpy()*10, cities_df['y'].to_numpy()*10)
cities_df['lat'] = lats
cities_df['lon'] = lons
cities_df = cities_df[['שם יישוב', 'lat', 'lon', 'תעתיק', 'סמל יישוב']]
cities_df.rename(columns = {"שם יישוב":"City", "תעתיק":"CityEng"} ,inplace=True)
cities_df.rename(columns={'סמל יישוב':'town_code'}, inplace=True)
cities_df.head(10)

,City,lat,lon,CityEng,town_code
0,אבו ג'ווייעד (שבט),31.230437,35.042204,ABU JUWEI'ID,967
1,אבו גוש,31.805999,35.110062,ABU GHOSH,472
2,אבו סנאן,32.960558,35.168216,ABU SINAN,473
3,אבו סריחאן (שבט),31.261641,34.858410,ABU SUREIHAN,935
4,אבו עבדון (שבט),31.302184,34.842506,ABU ABDUN,958
5,אבו עמאר (שבט),31.194207,34.947838,ABU AMMAR,1042
6,אבו עמרה (שבט),31.230031,34.842782,ABU AMRE,932
7,אבו קורינאת (יישוב),31.140570,34.962243,ABU QUREINAT,1342
8,אבו קורינאת (שבט),31.131112,34.968978,ABU QUREINAT,968
9,אבו רובייעה (שבט),31.239469,35.052686,ABU RUBEI'A,966


Merge:

In [10]:
# print(confirmed_cases.index)
confirmed_df = geo_data_processed.copy()
confirmed_df = confirmed_df.merge(cities_df, how='left', on="town_code")
# display(confirmed_df)
# confirmed_df.dropna(axis=0, how='any', inplace=True)
confirmed_df = confirmed_df.reset_index(drop=True) #TODO: correct and search
# confirmed_df.head(20)
# print(confirmed_df.loc[confirmed_df.City.str.contains("בני ברק"),:])
#print(str(confirmed_df.iloc[10,1]))
confirmed_df.head(20)
# display(confirmed_df)
def show_cases_by_city(city_name):
    return confirmed_df[confirmed_df['City'].str.contains(city_name)]
interact(show_cases_by_city, city_name='בני ברק')
plt.show()

interactive(children=(Text(value='בני ברק', description='city_name'), Output()), _dom_classes=('widget-interac…

World Map:

In [11]:
world_map = folium.Map(location=[31.4,35], tiles="cartodbpositron", zoom_start=8, max_zoom = 15, min_zoom = 8)

for i in range(0,len(confirmed_df)):
#     print(confirmed_df.loc[i,:].to_list[-5])
#     print(confirmed_df.iloc[i,-5])
    folium.Circle(
        location=[confirmed_df.at[i,'lat'], confirmed_df.at[i,'lon']],
        fill=True,
#         radius=(int((np.log(500*confirmed_df.iloc[i,-1]+1.00001)))+0.2),
         radius=int(confirmed_df.iloc[i,-5]),
        color='red',
        fill_color='indigo',
        tooltip = "<meta http-equiv='content-type' content='text/html; charset=UTF-8' /><div style='margin: 0; background-color: black; color: white;'>"+
                    "<h5 style='text-align:center;font-weight: bold'>"+str(confirmed_df.at[i,'CityEng']) + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-5])+"</li>"+
                        "</ul></div>",
        ).add_to(world_map)

world_map

In [12]:
import vincent
import json
# from bokeh.plotting import figure, output_file, show
from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.embed import file_html

# plot = figure()
# plot.circle([1,2], [3,4])

# html = file_html(plot, CDN, "my plot")


# scatter_json = line.to_json()

# # Let's convert it to dict.
# scatter_dict = json.loads(scatter_json)

world_map = folium.Map(location=[31.4,35], tiles="cartodbpositron", zoom_start=8, max_zoom = 15, min_zoom = 8)

dates = pd.to_datetime(pd.Series(confirmed_df.columns)[1:-4])

for i in range(0,len(confirmed_df)):
    values = confirmed_df.loc[i,:].drop(labels=['town_code', 'City', 'lat', 'lon', 'CityEng']).to_list()
    series = pd.Series(values, index=dates)
#     dates = 
    scatter_chart = vincent.Line(series)
    w = 600
    scatter_chart.width = w
    scatter_chart.height = 175
    scatter_chart.axis_titles(x='Date', y='Accumulated cases')
    popup = folium.Popup(max_width=w+50)
    radius = int(values[-1]) if not np.isnan(values[-1]) else 0
    folium.Vega(scatter_chart, height=225, width=w+50).add_to(popup)
    folium.Circle(
        location=[confirmed_df.at[i,'lat'], confirmed_df.at[i,'lon']],
        fill=True,
#         radius=(int((np.log(500*confirmed_df.iloc[i,-1]+1.00001)))+0.2),
         radius= radius ,
        color='red',
        fill_color='indigo',
        popup = popup,
        ).add_to(world_map)

world_map
world_map.save('map_timeseries20200621.html')
#https://towardsdatascience.com/building-covid-19-analysis-dashboard-using-python-and-voila-ee091f65dcbb